<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spell/F_spell_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본		    G_inc_again_so_model.pt
 산업분류자동화    'My Drive'
'Colab Notebooks'   Screenshot_20220401-001411_Chrome.jpg


In [11]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [12]:
df=df[(df['digit_1'] =='F')] #조건에 맞는 데이터 출력

In [13]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000010,F,42,423,영업장에서,고객의뢰를 받아,내부전기공사
1,id_0000052,F,42,422,사업장에서,의뢰.요청받아,상수도배관공사
2,id_0000197,F,41,411,현장에서,고개의 요청에따라,주택건설
3,id_0000222,F,42,422,건축물내,"기계,장비설치공사",공조기 설치
4,id_0000227,F,42,424,건물내,고객요구에 따라,인테리어공사
...,...,...,...,...,...,...,...
35045,id_0999849,F,42,424,NaN,토탈인테리어및 유리바닥재도소매,NaN
35046,id_0999875,F,41,412,건설현장에서,고객의 의뢰를 받아,토목공사
35047,id_0999989,F,41,412,현장에서,건설업,토목시설물
35048,id_0999993,F,41,412,고객의사업장에서,고객요구의따라,조경건설공사


In [14]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000010,F,42,423,영업장에서 고객의뢰를 받아 내부전기공사
1,id_0000052,F,42,422,사업장에서 의뢰.요청받아 상수도배관공사
2,id_0000197,F,41,411,현장에서 고개의 요청에따라 주택건설
3,id_0000222,F,42,422,"건축물내 기계,장비설치공사 공조기 설치"
4,id_0000227,F,42,424,건물내 고객요구에 따라 인테리어공사
...,...,...,...,...,...
35045,id_0999849,F,42,424,토탈인테리어및 유리바닥재도소매
35046,id_0999875,F,41,412,건설현장에서 고객의 의뢰를 받아 토목공사
35047,id_0999989,F,41,412,현장에서 건설업 토목시설물
35048,id_0999993,F,41,412,고객의사업장에서 고객요구의따라 조경건설공사


In [15]:
df['digit_3'].value_counts()

424    11152
422     5744
421     4724
423     4360
411     3582
412     3363
426     1566
425      559
Name: digit_3, dtype: int64

## 맞춤법 교정

In [16]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-8ybnqjg0
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-8ybnqjg0


In [17]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [18]:
df.to_csv('df_F_spellcheck.csv', index=False, encoding='utf-8-sig')